* This notebook uses data from https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/.
* It contains Experiment with LightGBM

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('data/dengue_features_train.csv')
#train_data['week_start_date'] = pd.to_datetime(train_data.week_start_date)
X_train = train_data.drop('week_start_date',axis=1)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   1456 non-null   object 
 1   year                                   1456 non-null   int64  
 2   weekofyear                             1456 non-null   int64  
 3   ndvi_ne                                1262 non-null   float64
 4   ndvi_nw                                1404 non-null   float64
 5   ndvi_se                                1434 non-null   float64
 6   ndvi_sw                                1434 non-null   float64
 7   precipitation_amt_mm                   1443 non-null   float64
 8   reanalysis_air_temp_k                  1446 non-null   float64
 9   reanalysis_avg_temp_k                  1446 non-null   float64
 10  reanalysis_dew_point_temp_k            1446 non-null   float64
 11  rean

In [3]:
labels = pd.read_csv('data/dengue_labels_train.csv')
labels.head()
merged = X_train.merge(labels,on=['city','year','weekofyear'])
y_train = merged['total_cases']
X_train = merged.drop('total_cases',axis=1)
X_train.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [4]:
X_train['year'] = X_train['year'].astype('object')
X_train['weekofyear'] = X_train['weekofyear'].astype('object')

X_train['week_year'] = X_train['city']+"_"+X_train['weekofyear'].astype('str')+"_"+X_train['year'].astype('str')
X_train = X_train.drop(['city','year','weekofyear'],axis=1)
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1455
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ndvi_ne                                1262 non-null   float64
 1   ndvi_nw                                1404 non-null   float64
 2   ndvi_se                                1434 non-null   float64
 3   ndvi_sw                                1434 non-null   float64
 4   precipitation_amt_mm                   1443 non-null   float64
 5   reanalysis_air_temp_k                  1446 non-null   float64
 6   reanalysis_avg_temp_k                  1446 non-null   float64
 7   reanalysis_dew_point_temp_k            1446 non-null   float64
 8   reanalysis_max_air_temp_k              1446 non-null   float64
 9   reanalysis_min_air_temp_k              1446 non-null   float64
 10  reanalysis_precip_amt_kg_per_m2        1446 non-null   float64
 11  rean

In [5]:
num_col = [col for col in X_train.columns if X_train[col].dtype=='int' or X_train[col].dtype=='float64']
cat_col = [col for col in X_train.columns if X_train[col].dtype=='object']


In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_processor = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('normalizer',Normalizer())
])
cat_processor = Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num',num_processor,num_col),
    ('cat',cat_processor,cat_col)
])


In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
model = XGBRegressor(n_estimators=5000,learning_rate=0.01)
estimators = [
     ('xgb', XGBRegressor(n_estimators=5000,learning_rate=0.01)),
     ('catboost', CatBoostRegressor())
 ]
reg = StackingRegressor(
     estimators=estimators,
 )
pipeline = Pipeline(steps=[
    ('processor',preprocessor),
    ('model',reg)
])

score = -1*cross_val_score(pipeline,X_train,y_train,cv=3,scoring='neg_mean_absolute_error')
print(f'MSE Loss mean = {np.mean(score)} max = {np.max(score)}')

Learning rate set to 0.038819
0:	learn: 18.9352355	total: 60.6ms	remaining: 1m
1:	learn: 18.7508256	total: 66ms	remaining: 32.9s
2:	learn: 18.5904891	total: 71.4ms	remaining: 23.7s
3:	learn: 18.4120068	total: 76.6ms	remaining: 19.1s
4:	learn: 18.2255596	total: 82ms	remaining: 16.3s
5:	learn: 18.1014428	total: 87.5ms	remaining: 14.5s
6:	learn: 17.9741768	total: 92.9ms	remaining: 13.2s
7:	learn: 17.7928498	total: 98.2ms	remaining: 12.2s
8:	learn: 17.7062603	total: 104ms	remaining: 11.4s
9:	learn: 17.5501158	total: 109ms	remaining: 10.8s
10:	learn: 17.4041720	total: 114ms	remaining: 10.3s
11:	learn: 17.2596396	total: 120ms	remaining: 9.84s
12:	learn: 17.1090204	total: 125ms	remaining: 9.48s
13:	learn: 17.0013853	total: 131ms	remaining: 9.19s
14:	learn: 16.8902517	total: 136ms	remaining: 8.92s
15:	learn: 16.7726058	total: 141ms	remaining: 8.69s
16:	learn: 16.6736295	total: 147ms	remaining: 8.48s
17:	learn: 16.5424007	total: 152ms	remaining: 8.3s
18:	learn: 16.4341680	total: 157ms	remaining

157:	learn: 12.4041321	total: 778ms	remaining: 4.15s
158:	learn: 12.3966111	total: 784ms	remaining: 4.15s
159:	learn: 12.3881922	total: 790ms	remaining: 4.14s
160:	learn: 12.3788593	total: 795ms	remaining: 4.14s
161:	learn: 12.3668403	total: 799ms	remaining: 4.13s
162:	learn: 12.3382362	total: 804ms	remaining: 4.13s
163:	learn: 12.3278622	total: 808ms	remaining: 4.12s
164:	learn: 12.3156638	total: 812ms	remaining: 4.11s
165:	learn: 12.3083105	total: 816ms	remaining: 4.1s
166:	learn: 12.2987735	total: 821ms	remaining: 4.09s
167:	learn: 12.2890666	total: 825ms	remaining: 4.09s
168:	learn: 12.2796733	total: 830ms	remaining: 4.08s
169:	learn: 12.2713676	total: 834ms	remaining: 4.07s
170:	learn: 12.2611499	total: 839ms	remaining: 4.07s
171:	learn: 12.2499648	total: 843ms	remaining: 4.06s
172:	learn: 12.2419799	total: 848ms	remaining: 4.05s
173:	learn: 12.2338077	total: 852ms	remaining: 4.04s
174:	learn: 12.2144803	total: 857ms	remaining: 4.04s
175:	learn: 12.1701075	total: 861ms	remaining: 

324:	learn: 10.7618681	total: 1.53s	remaining: 3.18s
325:	learn: 10.7554197	total: 1.54s	remaining: 3.18s
326:	learn: 10.7490489	total: 1.54s	remaining: 3.17s
327:	learn: 10.7428709	total: 1.54s	remaining: 3.17s
328:	learn: 10.7367283	total: 1.55s	remaining: 3.16s
329:	learn: 10.7306062	total: 1.55s	remaining: 3.15s
330:	learn: 10.6945685	total: 1.56s	remaining: 3.15s
331:	learn: 10.6884419	total: 1.56s	remaining: 3.15s
332:	learn: 10.6823602	total: 1.57s	remaining: 3.14s
333:	learn: 10.6761153	total: 1.57s	remaining: 3.13s
334:	learn: 10.6700980	total: 1.58s	remaining: 3.13s
335:	learn: 10.6641067	total: 1.58s	remaining: 3.13s
336:	learn: 10.6318228	total: 1.58s	remaining: 3.12s
337:	learn: 10.6258690	total: 1.59s	remaining: 3.11s
338:	learn: 10.6199405	total: 1.59s	remaining: 3.11s
339:	learn: 10.6140272	total: 1.6s	remaining: 3.1s
340:	learn: 10.6081589	total: 1.6s	remaining: 3.1s
341:	learn: 10.6023138	total: 1.61s	remaining: 3.09s
342:	learn: 10.5964951	total: 1.61s	remaining: 3.0

487:	learn: 9.3662781	total: 2.28s	remaining: 2.4s
488:	learn: 9.3623070	total: 2.29s	remaining: 2.39s
489:	learn: 9.3583415	total: 2.29s	remaining: 2.39s
490:	learn: 9.3543875	total: 2.3s	remaining: 2.38s
491:	learn: 9.3504368	total: 2.3s	remaining: 2.38s
492:	learn: 9.3464912	total: 2.31s	remaining: 2.37s
493:	learn: 9.3425573	total: 2.31s	remaining: 2.37s
494:	learn: 9.3216940	total: 2.32s	remaining: 2.36s
495:	learn: 9.3177617	total: 2.32s	remaining: 2.36s
496:	learn: 9.3139012	total: 2.33s	remaining: 2.35s
497:	learn: 9.3099976	total: 2.33s	remaining: 2.35s
498:	learn: 9.3060992	total: 2.33s	remaining: 2.34s
499:	learn: 9.2852915	total: 2.34s	remaining: 2.34s
500:	learn: 9.2813806	total: 2.34s	remaining: 2.33s
501:	learn: 9.2774855	total: 2.35s	remaining: 2.33s
502:	learn: 9.2736091	total: 2.35s	remaining: 2.32s
503:	learn: 9.2697500	total: 2.36s	remaining: 2.32s
504:	learn: 9.2658999	total: 2.36s	remaining: 2.31s
505:	learn: 9.2620545	total: 2.37s	remaining: 2.31s
506:	learn: 9.2

660:	learn: 8.3385735	total: 3.06s	remaining: 1.57s
661:	learn: 8.3356404	total: 3.06s	remaining: 1.56s
662:	learn: 8.3327646	total: 3.07s	remaining: 1.56s
663:	learn: 8.3298455	total: 3.07s	remaining: 1.55s
664:	learn: 8.3269793	total: 3.08s	remaining: 1.55s
665:	learn: 8.3240723	total: 3.08s	remaining: 1.54s
666:	learn: 8.3048294	total: 3.08s	remaining: 1.54s
667:	learn: 8.3019144	total: 3.09s	remaining: 1.53s
668:	learn: 8.2990106	total: 3.09s	remaining: 1.53s
669:	learn: 8.2961121	total: 3.1s	remaining: 1.53s
670:	learn: 8.2932197	total: 3.1s	remaining: 1.52s
671:	learn: 8.2903384	total: 3.11s	remaining: 1.52s
672:	learn: 8.2874616	total: 3.11s	remaining: 1.51s
673:	learn: 8.2846314	total: 3.12s	remaining: 1.51s
674:	learn: 8.2714663	total: 3.12s	remaining: 1.5s
675:	learn: 8.2685960	total: 3.12s	remaining: 1.5s
676:	learn: 8.2657779	total: 3.13s	remaining: 1.49s
677:	learn: 8.2629226	total: 3.13s	remaining: 1.49s
678:	learn: 8.2597741	total: 3.14s	remaining: 1.48s
679:	learn: 8.25

...

181:	learn: 31.1623165	total: 703ms	remaining: 3.16s
182:	learn: 31.1227541	total: 707ms	remaining: 3.15s
183:	learn: 31.0861513	total: 711ms	remaining: 3.15s
184:	learn: 31.0560104	total: 714ms	remaining: 3.15s
185:	learn: 31.0113782	total: 718ms	remaining: 3.14s
186:	learn: 30.9682677	total: 722ms	remaining: 3.14s
187:	learn: 30.9312588	total: 726ms	remaining: 3.13s
188:	learn: 30.8951095	total: 730ms	remaining: 3.13s
189:	learn: 30.8581756	total: 734ms	remaining: 3.13s
190:	learn: 30.8213721	total: 738ms	remaining: 3.12s
191:	learn: 30.7860514	total: 742ms	remaining: 3.12s
192:	learn: 30.7500978	total: 745ms	remaining: 3.12s
193:	learn: 30.7069834	total: 749ms	remaining: 3.11s
194:	learn: 30.6663363	total: 753ms	remaining: 3.11s
195:	learn: 30.6243422	total: 756ms	remaining: 3.1s
196:	learn: 30.5891555	total: 760ms	remaining: 3.1s
197:	learn: 30.5544760	total: 764ms	remaining: 3.09s
198:	learn: 30.5132246	total: 768ms	remaining: 3.09s
199:	learn: 30.4731042	total: 772ms	remaining: 3

381:	learn: 25.1913567	total: 1.47s	remaining: 2.37s
382:	learn: 25.1716545	total: 1.47s	remaining: 2.37s
383:	learn: 25.1520197	total: 1.47s	remaining: 2.36s
384:	learn: 25.1324423	total: 1.48s	remaining: 2.36s
385:	learn: 25.1129797	total: 1.48s	remaining: 2.36s
386:	learn: 25.0936025	total: 1.49s	remaining: 2.35s
387:	learn: 25.0742623	total: 1.49s	remaining: 2.35s
388:	learn: 25.0549606	total: 1.49s	remaining: 2.34s
389:	learn: 25.0357541	total: 1.5s	remaining: 2.34s
390:	learn: 25.0165873	total: 1.5s	remaining: 2.34s
391:	learn: 24.9951592	total: 1.5s	remaining: 2.33s
392:	learn: 24.9760930	total: 1.51s	remaining: 2.33s
393:	learn: 24.9570520	total: 1.51s	remaining: 2.32s
394:	learn: 24.9380938	total: 1.51s	remaining: 2.32s
395:	learn: 24.9191785	total: 1.52s	remaining: 2.32s
396:	learn: 24.7990675	total: 1.52s	remaining: 2.31s
397:	learn: 24.6668130	total: 1.53s	remaining: 2.31s
398:	learn: 24.6478052	total: 1.53s	remaining: 2.3s
399:	learn: 24.6290294	total: 1.53s	remaining: 2.3

581:	learn: 20.6693822	total: 2.23s	remaining: 1.6s
582:	learn: 20.6575014	total: 2.23s	remaining: 1.6s
583:	learn: 20.6456697	total: 2.24s	remaining: 1.59s
584:	learn: 20.6338587	total: 2.24s	remaining: 1.59s
585:	learn: 20.6220720	total: 2.24s	remaining: 1.58s
586:	learn: 20.6103075	total: 2.25s	remaining: 1.58s
587:	learn: 20.5985732	total: 2.25s	remaining: 1.58s
588:	learn: 20.5868598	total: 2.25s	remaining: 1.57s
589:	learn: 20.5751791	total: 2.26s	remaining: 1.57s
590:	learn: 20.5635230	total: 2.26s	remaining: 1.57s
591:	learn: 20.5504410	total: 2.27s	remaining: 1.56s
592:	learn: 20.5388183	total: 2.27s	remaining: 1.56s
593:	learn: 20.5272119	total: 2.27s	remaining: 1.55s
594:	learn: 20.5156334	total: 2.28s	remaining: 1.55s
595:	learn: 20.5040753	total: 2.28s	remaining: 1.55s
596:	learn: 20.4911950	total: 2.29s	remaining: 1.54s
597:	learn: 20.4796913	total: 2.29s	remaining: 1.54s
598:	learn: 20.4682041	total: 2.29s	remaining: 1.53s
599:	learn: 20.4567383	total: 2.3s	remaining: 1.

778:	learn: 17.5304719	total: 2.99s	remaining: 848ms
779:	learn: 17.5221919	total: 2.99s	remaining: 844ms
780:	learn: 17.5139292	total: 3s	remaining: 840ms
781:	learn: 17.5056861	total: 3s	remaining: 836ms
782:	learn: 17.4974756	total: 3s	remaining: 833ms
783:	learn: 17.4892800	total: 3.01s	remaining: 829ms
784:	learn: 17.4811027	total: 3.01s	remaining: 825ms
785:	learn: 17.4729444	total: 3.02s	remaining: 821ms
786:	learn: 17.4648012	total: 3.02s	remaining: 817ms
787:	learn: 17.4566715	total: 3.02s	remaining: 814ms
788:	learn: 17.4485571	total: 3.03s	remaining: 810ms
789:	learn: 17.4404645	total: 3.03s	remaining: 806ms
790:	learn: 17.4323861	total: 3.04s	remaining: 802ms
791:	learn: 17.4243230	total: 3.04s	remaining: 798ms
792:	learn: 17.4162815	total: 3.04s	remaining: 794ms
793:	learn: 17.4082541	total: 3.05s	remaining: 790ms
794:	learn: 17.3890338	total: 3.05s	remaining: 787ms
795:	learn: 17.3810113	total: 3.05s	remaining: 783ms
796:	learn: 17.3730060	total: 3.06s	remaining: 779ms
79

966:	learn: 15.2892530	total: 3.73s	remaining: 127ms
967:	learn: 15.2832561	total: 3.74s	remaining: 124ms
968:	learn: 15.2772825	total: 3.74s	remaining: 120ms
969:	learn: 15.2714219	total: 3.75s	remaining: 116ms
970:	learn: 15.2654741	total: 3.75s	remaining: 112ms
971:	learn: 15.2458200	total: 3.75s	remaining: 108ms
972:	learn: 15.2398670	total: 3.76s	remaining: 104ms
973:	learn: 15.2339244	total: 3.76s	remaining: 100ms
974:	learn: 15.2006392	total: 3.77s	remaining: 96.6ms
975:	learn: 15.1946961	total: 3.77s	remaining: 92.7ms
976:	learn: 15.1888681	total: 3.77s	remaining: 88.8ms
977:	learn: 15.1829605	total: 3.78s	remaining: 85ms
978:	learn: 15.1560026	total: 3.78s	remaining: 81.1ms
979:	learn: 15.1500950	total: 3.78s	remaining: 77.2ms
980:	learn: 15.1331351	total: 3.79s	remaining: 73.4ms
981:	learn: 15.1272337	total: 3.79s	remaining: 69.5ms
982:	learn: 15.1213542	total: 3.79s	remaining: 65.6ms
983:	learn: 15.1154905	total: 3.8s	remaining: 61.8ms
984:	learn: 15.1097416	total: 3.8s	rema

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [37]:
model = pipeline.fit(X_train,y_train)

In [50]:
X_test = pd.read_csv('data/dengue_features_test.csv')
X_test.head()
X_test = X_test.drop('week_start_date',axis=1)
X_test['year'] = X_test['year'].astype('object')
X_test['weekofyear'] = X_test['weekofyear'].astype('object')

X_test['week_year'] = X_test['city']+"_"+X_test['weekofyear'].astype('str')+"_"+X_test['year'].astype('str')
X_test = X_test.drop(['city','year','weekofyear'],axis=1)

In [51]:
X_test['total_cases'] = model.predict(X_test)

In [52]:
X_test.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,week_year,total_cases
0,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,...,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2,sj_18_2008,14.678601
1,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,...,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3,sj_19_2008,11.341932
2,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,...,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0,sj_20_2008,10.495125
3,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,...,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3,sj_21_2008,37.485123
4,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,...,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1,sj_22_2008,32.415176


In [53]:
submission = pd.DataFrame(X_test.week_year.str.split('_').tolist(),
                                 columns = ['city','weekofyear','year'])

In [56]:
submission['total_cases'] = X_test['total_cases'].astype('int64')
submission = submission[['city','year','weekofyear','total_cases']]
submission

,city,year,weekofyear,total_cases
0,sj,2008,18,14
1,sj,2008,19,11
2,sj,2008,20,10
3,sj,2008,21,37
4,sj,2008,22,32
...,...,...,...,...
411,iq,2013,22,4
412,iq,2013,23,5
413,iq,2013,24,4
414,iq,2013,25,3


In [57]:
submission.to_csv('submission.csv', index=False)